# Procedure for estimating noise parameters

In [1]:
from matplotlib import pyplot as plt
from matplotlib.figure import figaspect

import numpy as np
from numpy.random import default_rng

import thztools as thz

## Simulate measurements
Simulate a set of `m` waveforms, each sampled at `n` time points, with noise parameters
`sigma_alpha`, `sigma_beta`, and `sigma_tau`, and store them in an array `x`. Note that
`x` stores the waveforms in row orientation, with shape `(m, n)`, because NumPy
broadcasting rules and FFT functions are simpler for arrays that are row-oriented.
Measurement waveforms are typically loaded from data files in column orientation,
however, so the `tdnoisefit` function assumes that the data array is column-oriented—this
may be worth changing in a future version.

In [2]:
n = 256  # Number of samples
m = 50  # Number of waveforms
ts = 0.05  # Sampling time [ps]
t0 = 2.5  # Peak pulse time [ps]
fs = 1 / ts  # Sampling frequency [THz]

sigma_alpha = 1e-4  # Additive noise amplitude [signal units]
sigma_beta = 1e-2  # Multiplicative noise amplitude [dimensionless]
sigma_tau = 1e-3  # Time base noise amplitude [ps]
sigma_parms = np.array([sigma_alpha, sigma_beta, sigma_tau])

seed = 0  # RNG seed
rng = default_rng(seed)

In [3]:
mu, t = thz.thzgen(n, ts, t0)
sigma = thz.noiseamp(sigma_parms, mu, ts)

# Use broadcasting to generate multiple simulated noisy measurements of y
x = mu + sigma * rng.standard_normal((m, n))

## Fit for the noise parameters

Set the initial guesses `a0` and `eta0` for the amplitudes and delays, respectively, and
estimate the noise parameters from the simulated measurements. Note that we must take the
transpose of `x`, to convert it from row-orientation to column-orientation.

In [4]:
# tdnoisefit expects a column-oriented data array
a0 = np.ones(m)
eta0 = np.zeros(m)
result = thz.tdnoisefit(
    x.T, a0=a0, eta0=eta0, ts=ts, fix_a=False, fix_eta=False
)

### Check the variance amplitude estimates

The `tdnoisefit` function returns a tuple with three elements: a dictionary `p` of output
parameters; the value `fval` of the cost function at those parameters; and a dictionary
`diagnostic` that includes additional diagnostic information about the result, including
the parameter uncertainty. Note that noise parameter uncertainty is expressed as a
standard deviation, so the uncertainty in the noise parameters refers to the standard
deviation of the variance amplitude. We apply the Poisson correction to each variance
parameter and its associated uncertainty—that is we multiply each by `m / (m - 1)`.

In [5]:
p_names = ["var_alpha", "var_beta", "var_tau"]
for i in range(3):
    val = result[0]['var'][i] * (m / (m - 1))
    err = result[2]['err']['var'][i, i] * (m / (m - 1))
    print(f"Estimated {p_names[i]}: {val:.4g} ± {err:.4g}")

Estimated var_alpha: 9.961e-09 ± 1.505e-10
Estimated var_beta: 9.437e-05 ± 3.555e-06
Estimated var_tau: 8.819e-07 ± 7.455e-08


### Check the noise amplitude estimates

Compare the noise amplitude estimates with their true values, specified in the
simulation. The noise amplitudes are just the square root of the variance amplitudes,
and propagating the error for the noise amplitude uncertainty yields

$$
\sigma^2_{\sigma_k} = \left(\frac{dV_k}{d\sigma_k}\right)^{-2} \sigma^2_{V_k}
= \frac{1}{2\sigma_k} \sigma^2_{V_k}.
$$

In [6]:
p_names = ["sigma_alpha", "sigma_beta", "sigma_tau"]
for i in range(3):
    val = np.sqrt(result[0]['var'][i] * (m / (m - 1)))
    err = 0.5 * result[2]['err']['var'][i, i] * (m / (m - 1)) / val
    print(f"Estimated {p_names[i]}: {val:.4g} ± {err:.4g}")
for i in range(3):
    val = np.sqrt(result[0]['var'][i] * (m / (m - 1))) / sigma_parms[i]
    err = (
        0.5
        * result[2]['err']['var'][i, i]
        * (m / (m - 1))
        / (sigma_parms[i] ** 2)
    )
    print(
        f"Estimated {p_names[i]}, relative to true value: {val:.3f} ± {err:.3f}"
    )

Estimated sigma_alpha: 9.981e-05 ± 7.541e-07
Estimated sigma_beta: 0.009715 ± 0.000183
Estimated sigma_tau: 0.0009391 ± 3.97e-05
Estimated sigma_alpha, relative to true value: 0.998 ± 0.008
Estimated sigma_beta, relative to true value: 0.971 ± 0.018
Estimated sigma_tau, relative to true value: 0.939 ± 0.037
